<a href="https://colab.research.google.com/github/hasanm08/ai-notebooks/blob/main/NaiveBayes_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loading Data

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip
!unzip '/content/sentiment labelled sentences.zip'

--2021-12-14 20:18:52--  https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84188 (82K) [application/x-httpd-php]
Saving to: ‘sentiment labelled sentences.zip.1’

sentiment labelled  100%[===================>]  82.21K  --.-KB/s    in 0.1s    

2021-12-14 20:18:52 (643 KB/s) - ‘sentiment labelled sentences.zip.1’ saved [84188/84188]

Archive:  /content/sentiment labelled sentences.zip
replace sentiment labelled sentences/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


## Tokenization and cleanup

In [ ]:
import nltk
nltk.download("punkt")
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
import random
def clean_text(file_path):
    with open(file_path) as f:
      texts = f.readlines()
    x=[]
    y=[]
    for text in texts:
      y.append(int((text.split('\t')[1]).replace("\n","")))
      tokens = word_tokenize(text)
      # Remove the punctuations
      tokens = [word for word in tokens if word.isalpha()]
      # Lower the tokens
      tokens = [word.lower() for word in tokens]
      # Remove stopword
      tokens = [word for word in tokens if not word in stopwords.words("english")]
      # Lemmatize
      lemma = WordNetLemmatizer()
      tokens = [lemma.lemmatize(word, pos = "v") for word in tokens]
      tokens = [lemma.lemmatize(word, pos = "n") for word in tokens]
      x.append(tokens)
    x_train=x[int(0.8*len(x)):]
    y_train=y[int(0.8*len(y)):]
    x_test=x[:int(0.2*len(x))]
    y_test=y[:int(0.2*len(y))]
    return x_train,y_train,x_test,y_test

In [ ]:
imdb_x_train,imdb_y_train,imdb_x_test,imdb_y_test= clean_text('/content/sentiment labelled sentences/imdb_labelled.txt')
amazon_x_train,amazon_y_train,amazon_x_test,amazon_y_test=clean_text('/content/sentiment labelled sentences/amazon_cells_labelled.txt')
yelp_x_train,yelp_y_train,yelp_x_test,yelp_y_test=clean_text('/content/sentiment labelled sentences/yelp_labelled.txt')


## dictionary

In [ ]:
import pandas as pd
def get_count_df(X, y):
  from collections import defaultdict

  positive_dic = defaultdict(int)
  negative_dic = defaultdict(int)

  # Count each word in every sentence
  for sentence, label in zip(X, y):
    # If the sentence's label is 0 increment it's count_neg
    if label==0:
      for word in sentence:
        negative_dic[word]+=1

    # If the sentence's label is 1 increment it's count_pos
    if label==1:
      for word in sentence:
        positive_dic[word]+=1

  # Convert dicts to dataframes
  df_pos = pd.DataFrame({"word":positive_dic.keys(), "count_pos":positive_dic.values()})
  df_neg = pd.DataFrame({"word":negative_dic.keys(), "count_neg":negative_dic.values()})
  
  # Join dataframes
  df = pd.merge(df_pos, df_neg, on='word', how='outer')
  
  # Sort df
  df = df.sort_values(by=["count_pos","count_neg"], ascending=False)

  df = df.fillna(0)

  return df

In [ ]:
def probability(word, df):
  # calculate totals
  total_pos = df["count_pos"].sum()
  total_neg = df["count_neg"].sum()
  total = total_pos + total_neg
  p_pos=(total_pos)/(total)
  p_neg=(total_neg)/(total)

  # get counts for 'word'
  pos = int(df.loc[df.word == word]["count_pos"])
  neg = int(df.loc[df.word == word]["count_neg"])
  # calculate probabilities
  p_word = (pos+neg) / (total)
  p_word_pos = (pos)/ (total_pos)
  p_word_neg = (neg) / (total_neg)

  # print probabilities
  # print(f"p({word}) = {p_word}")
  # print(f"p({word}|pos) = {p_word_pos}")
  # print(f"p({word}|neg) = {p_word_neg}")
  return p_word_pos,p_pos,p_word_neg,p_neg

In [ ]:
import numpy as np
def naive(x,df):
  res=[]
  for sent in x:
    predict_pos=1
    predict_neg=1
    p_neg=1
    p_pos=1
    for word in sent:
      a,b,c,d=probability(word, df)
      p_pos=b
      p_neg=d
      predict_pos=predict_pos*a
      predict_neg=predict_neg*c
    predict_neg=predict_neg*p_neg
    predict_pos=predict_pos*p_pos
    predict_pos=np.log(predict_pos)
    predict_neg=np.log(predict_neg)
    if(predict_pos>predict_neg):
      res.append(1)
    else:
      res.append(0)
  return res

In [ ]:
def accuracy(y, y_hat):
    diff=np.array(y)-np.array(y_hat)
    andd=np.array(y)+np.array(y_hat)
    tp=np.sum(andd==2)
    tn=np.sum(andd==0)
    fp=np.sum(diff==-1)
    fn=np.sum(diff==1)
    print('     pos',' ','neg')
    print("pos ",tp," ",fp)
    print("neg ",fn,"   ",tn)
    
    print("pos acc","\t",tp/(tp+fp) )
    print('neg acc','\t',tn/(tn+fn) )
    return np.mean(np.array(y)==np.array(y_hat)) #TP+TN/TP+TN+FP+FN

In [ ]:
imdb_train=naive(imdb_x_train,get_count_df(imdb_x_train,imdb_y_train))
imdb_test=naive(imdb_x_test,get_count_df(imdb_x_test,imdb_y_test))
amazon_train=naive(amazon_x_train,get_count_df(amazon_x_train,amazon_y_train))
amazon_test=naive(amazon_x_test,get_count_df(amazon_x_test,amazon_y_test))
yelp_train=naive(yelp_x_train,get_count_df(yelp_x_train,yelp_y_train))
yelp_test=naive(yelp_x_test,get_count_df(yelp_x_test,yelp_y_test))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in log


In [ ]:
print("imdb")
print(f"\nacc \ttrain:{accuracy(imdb_y_train, imdb_train)}\ttest:{accuracy(imdb_y_test, imdb_test)}\n")
print("amazon")
print(f"\nacc \ttrain:{accuracy(amazon_y_train, amazon_train)}\ttest:{accuracy(amazon_y_test, amazon_test)}\n")
print("\nyelp")
print(f"acc \ttrain:{accuracy(yelp_y_train, yelp_train)}\ttest:{accuracy(yelp_y_test, yelp_test)}\n")

imdb
     pos   neg
pos  113   1
neg  1     85
pos acc 	 0.9912280701754386
neg acc 	 0.9883720930232558
     pos   neg
pos  78   1
neg  4     117
pos acc 	 0.9873417721518988
neg acc 	 0.9669421487603306

acc 	train:0.99	test:0.975

amazon
     pos   neg
pos  90   3
neg  1     106
pos acc 	 0.967741935483871
neg acc 	 0.9906542056074766
     pos   neg
pos  101   2
neg  0     97
pos acc 	 0.9805825242718447
neg acc 	 1.0

acc 	train:0.98	test:0.99


yelp
     pos   neg
pos  48   0
neg  0     152
pos acc 	 1.0
neg acc 	 1.0
     pos   neg
pos  111   1
neg  1     87
pos acc 	 0.9910714285714286
neg acc 	 0.9886363636363636
acc 	train:1.0	test:0.99



In [ ]:
def prob_dict(df):
  df["prob_pos"] = df["count_pos"]/(df["count_pos"] + df["count_neg"])
  df["prob_neg"] = df["count_neg"]/(df["count_pos"] + df["count_neg"])
  return df.drop(columns=["count_pos", "count_neg"])

In [ ]:
imdb = get_count_df(imdb_x_train, imdb_y_train)
amazon=get_count_df(amazon_x_train, amazon_y_train)
yelp=get_count_df(yelp_x_train, yelp_y_train)
display(prob_dict(imdb))
display(prob_dict(amazon))
display(prob_dict(yelp))


,word,prob_pos,prob_neg
1,film,0.760870,0.239130
42,movie,0.543478,0.456522
93,character,0.812500,0.187500
55,one,0.391304,0.608696
143,like,0.500000,0.500000
...,...,...,...
847,regrettable,0.000000,1.000000
848,embarrass,0.000000,1.000000
849,exceptionally,0.000000,1.000000
850,intelligence,0.000000,1.000000


,word,prob_pos,prob_neg
0,good,0.777778,0.222222
18,phone,0.392857,0.607143
1,work,0.550000,0.450000
12,headset,1.000000,0.000000
34,product,0.466667,0.533333
...,...,...,...
453,red,0.000000,1.000000
454,port,0.000000,1.000000
455,irda,0.000000,1.000000
456,answer,0.000000,1.000000


,word,prob_pos,prob_neg
59,great,1.000000,0.000000
3,place,0.285714,0.714286
0,service,0.350000,0.650000
55,good,0.555556,0.444444
29,love,0.714286,0.285714
...,...,...,...
592,life,0.000000,1.000000
593,pour,0.000000,1.000000
594,salt,0.000000,1.000000
595,wind,0.000000,1.000000
